In [26]:
%reload_ext autoreload 
%autoreload 2
from cchess import *
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import random 
import time
from utils import Dataset,ProgressBar
from tflearn.data_flow import DataFlow,DataFlowStatus,FeedDictFlow
from tflearn.data_utils import Preloader,ImagePreloader
import scipy
import pandas as pd
import xmltodict
from game_convert import convert_game
import tflearn
from gameplay import *

In [254]:
GPU_CORE = 1
BATCH_SIZE = 1
BEGINING_LR = 0.01
#TESTIMG_WIDTH = 500
model_name = '11_4_resnet'
data_dir = 'data/imsa-cbf/'

In [19]:
def res_block(inputx,name,training,block_num=2,filters=256,kernel_size=(3,3)):
    net = inputx
    for i in range(block_num):
        net = tf.layers.conv2d(net,filters=filters,kernel_size=kernel_size,activation=None,name="{}_res_conv{}".format(name,i),padding='same')
        net = tf.layers.batch_normalization(net,training=training,name="{}_res_bn{}".format(name,i))
        if i == block_num - 1:
            net = net + inputx #= tf.concat((inputx,net),axis=-1)
        net = tf.nn.elu(net,name="{}_res_elu{}".format(name,i))
    return net

def conv_block(inputx,name,training,block_num=1,filters=2,kernel_size=(1,1)):
    net = inputx
    for i in range(block_num):
        net = tf.layers.conv2d(net,filters=filters,kernel_size=kernel_size,activation=None,name="{}_convblock_conv{}".format(name,i),padding='same')
        net = tf.layers.batch_normalization(net,training=training,name="{}_convblock_bn{}".format(name,i))
        net = tf.nn.elu(net,name="{}_convblock_elu{}".format(name,i))
    # net [None,10,9,2]
    netshape = net.get_shape().as_list()
    print("inside conv block {}".format(str(netshape)))
    net = tf.reshape(net,shape=(-1,netshape[1] * netshape[2] * netshape[3]))
    net = tf.layers.dense(net,10 * 9,name="{}_dense".format(name))
    net = tf.nn.elu(net,name="{}_elu".format(name))
    return net

def res_net_board(inputx,name,training,filters=256):
    net = inputx
    net = tf.layers.conv2d(net,filters=filters,kernel_size=(3,3),activation=None,name="{}_res_convb".format(name),padding='same')
    net = tf.layers.batch_normalization(net,training=training,name="{}_res_bnb".format(name))
    net = tf.nn.elu(net,name="{}_res_elub".format(name))
    for i in range(NUM_RES_LAYERS):
        net = res_block(net,name="{}_layer_{}".format(name,i + 1),training=training)
        print(net.get_shape().as_list())
    print("inside res net {}".format(str(net.get_shape().as_list())))
    net_unsoftmax = conv_block(net,name="{}_conv".format(name),training=training)
    return net_unsoftmax

def get_scatter(name):
    with tf.variable_scope("Test"):
        ph = tf.placeholder(tf.float32,name=name)
        op = tf.summary.scalar(name,ph)
    return ph,op

In [6]:
tf.reset_default_graph()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True
sess = tf.Session(config=config)

NUM_RES_LAYERS = 10

with tf.device("/gpu:{}".format(GPU_CORE)):
    X1 = tf.placeholder(tf.float32,[None,10,9,14])
    y1 = tf.placeholder(tf.float32,[None,10,9])
    X2 = tf.placeholder(tf.float32,[None,10,9,15])
    y2 = tf.placeholder(tf.float32,[None,10,9])
    
    training = tf.placeholder(tf.bool,name='training_mode')
    learning_rate = tf.placeholder(tf.float32)
    global_step = tf.train.get_or_create_global_step()
    
    net_unsoftmax1 = res_net_board(X1,"selectnet",training=training)
    net_unsoftmax2 = res_net_board(X2,"movenet",training=training)
    
    target1 = tf.reshape(y1,(-1,10 * 9))
    target2 = tf.reshape(y2,(-1,10 * 9))
    with tf.variable_scope("Loss"):
        loss_select = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=target1,logits=net_unsoftmax1))
        loss_move = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=target2,logits=net_unsoftmax2))
        loss = loss_select + loss_move
        
        loss_select_summary = tf.summary.scalar("loss_select",loss_select)
        loss_move_summary = tf.summary.scalar("loss_move",loss_move)
        loss_summary = tf.summary.scalar("total_loss",loss)
    net_softmax1 = tf.nn.softmax(net_unsoftmax1)
    net_softmax2 = tf.nn.softmax(net_unsoftmax2)
    
    correct_prediction1 = tf.equal(tf.argmax(target1,1), tf.argmax(net_softmax1,1))
    correct_prediction2 = tf.equal(tf.argmax(target2,1), tf.argmax(net_softmax2,1))
    
    with tf.variable_scope("Accuracy"):
        accuracy_select = tf.reduce_mean(tf.cast(correct_prediction1, tf.float32))
        accuracy_move = tf.reduce_mean(tf.cast(correct_prediction2, tf.float32))
        accuracy_total = accuracy_select * accuracy_move
        
        acc_select_summary = tf.summary.scalar("accuracy_select",accuracy_select)
        acc_move_summary = tf.summary.scalar("accuracy_move",accuracy_move)
        acc_summary = tf.summary.scalar("acc_summary",accuracy_total)
        
    summary_op = tf.summary.merge([loss_select_summary,loss_move_summary,loss_summary
                                  ,acc_select_summary,acc_move_summary,acc_summary])
    
    test_select,test_select_summary = get_scatter("test_select_loss")
    test_move,test_move_summary = get_scatter("test_move_loss")
    test_total,test_total_summary = get_scatter("test_total_loss")
    test_selectacc,test_selectacc_summary = get_scatter("test_select_acc")
    test_moveacc,test_moveacc_summary = get_scatter("test_move_acc")
    test_totalacc,test_totalacc_summary = get_scatter("test_total_acc")
    
    test_summary_op = tf.summary.merge([test_select_summary,test_move_summary,test_total_summary
                                       ,test_selectacc_summary,test_moveacc_summary,test_totalacc_summary])
    
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=0.9)
        train_op = optimizer.minimize(loss,global_step=global_step)

    train_summary_writer = tf.summary.FileWriter("./log/{}_train".format(model_name), sess.graph)

[None, 10, 9, 256]
[None, 10, 9, 256]
[None, 10, 9, 256]
[None, 10, 9, 256]
[None, 10, 9, 256]
[None, 10, 9, 256]
[None, 10, 9, 256]
[None, 10, 9, 256]
[None, 10, 9, 256]
[None, 10, 9, 256]
inside res net [None, 10, 9, 256]
inside conv block [None, 10, 9, 2]
[None, 10, 9, 256]
[None, 10, 9, 256]
[None, 10, 9, 256]
[None, 10, 9, 256]
[None, 10, 9, 256]
[None, 10, 9, 256]
[None, 10, 9, 256]
[None, 10, 9, 256]
[None, 10, 9, 256]
[None, 10, 9, 256]
inside res net [None, 10, 9, 256]
inside conv block [None, 10, 9, 2]


In [7]:
saver = tf.train.Saver(var_list=tf.global_variables())
saver.restore(sess,"models/11_4_resnet/model_10")

INFO:tensorflow:Restoring parameters from models/11_4_resnet/model_10


In [20]:
class ElePreloader(object):
    def __init__(self,datafile,batch_size=64):
        self.batch_size=batch_size
        content = pd.read_csv(datafile,header=None,index_col=None)
        self.filelist = [i[0] for i in content.get_values()]
        self.pos = 0
        self.feature_list = {"red":['A', 'B', 'C', 'K', 'N', 'P', 'R']
                             ,"black":['a', 'b', 'c', 'k', 'n', 'p', 'r']}
        self.batch_size = batch_size
        self.batch_iter = self.__iter()
        assert(len(self.filelist) > batch_size)
        self.game_iterlist = [None for i in self.filelist]
    
    def __iter(self):
        retx1,rety1,retx2,rety2 = [],[],[],[]
        filelist = []
        while True:
            for i in range(self.batch_size):
                if self.game_iterlist[i] == None:
                    if len(filelist) == 0:
                        filelist = copy.copy(self.filelist)
                        random.shuffle(filelist)
                    self.game_iterlist[i] = convert_game(filelist.pop(),feature_list=self.feature_list)
                game_iter = self.game_iterlist[i]
                
                try:
                    x1,y1,x2,y2 = game_iter.__next__()
                    x1 = np.transpose(x1,[1,2,0])
                    x2 = np.transpose(x2,[1,2,0])
                    x1 = np.expand_dims(x1,axis=0)
                    x2 = np.expand_dims(x2,axis=0)
                    retx1.append(x1)
                    rety1.append(y1)
                    retx2.append(x2)
                    rety2.append(y2)
                    if len(retx1) >= self.batch_size:
                        yield (np.concatenate(retx1,axis=0),np.asarray(rety1)
                               ,np.concatenate(retx2,axis=0),np.asarray(rety2))
                        retx1,rety1,retx2,rety2 = [],[],[],[]
                except :
                    self.game_iterlist[i] = None

    def __getitem__(self, id):
        
        x1,y1,x2,y2 = self.batch_iter.__next__()
        return x1,y1,x2,y2
        
    def __len__(self):
        return 10000

In [21]:
testset = ElePreloader(datafile='data/test_list.csv',batch_size=BATCH_SIZE)
with tf.device("/gpu:{}".format(GPU_CORE)):
    coord = tf.train.Coordinator()
    testflow = FeedDictFlow({
            'data':testset,
        },coord,batch_size=1,shuffle=True,continuous=True,num_threads=1)
testflow.start()

In [22]:
sample_x1,sample_y1,sample_x2,sample_y2 = testflow.next()['data']

In [23]:
sample_x1.shape,sample_y1.shape,sample_x2.shape,sample_y2.shape

((1, 10, 9, 14), (1, 10, 9), (1, 10, 9, 15), (1, 10, 9))

In [25]:
accs = []
accselects = []
accmoves = []
losses = []
lossselects = []
lossmoves = []
pb = ProgressBar(worksum=300 * 256,info="validating ")
pb.startjob()
for one_batch in range(300 * 256):
    batch_x1,batch_y1,batch_x2,batch_y2 = testflow.next()['data']
    step_loss_move,step_loss_select,step_loss,step_accuracy_move,step_accuracy_select,step_accuracy_total = sess.run(
        [loss_move,loss_select,loss,accuracy_move,accuracy_select,accuracy_total],feed_dict={
            X1:batch_x1,y1:batch_y1,X2:batch_x2,y2:batch_y2,training:False
        })
    accs.append(step_accuracy_total)
    accselects.append(step_accuracy_select)
    accmoves.append(step_accuracy_move)
    losses.append(step_loss)
    lossselects.append(step_loss_select)
    lossmoves.append(step_loss_move)
    
    pb.complete(BATCH_SIZE)
print("TEST ACC {} SELACC {} MOVACC {} LOSS {}".format(np.average(accs),np.average(accselects)
                                                       ,np.average(accmoves),np.average(losses)))

validating  100.00 % [==================================================>] 76800/76800 	 used:1461s eta:0 ssTEST ACC 0.45613279938697815 SELACC 0.5695963501930237 MOVACC 0.8004036545753479 LOSS 1.743126630783081


In [184]:
def randargmax(inputx):
    inputx = np.copy(inputx)
    indexs = np.asarray(list(range(len(inputx))))
    random.shuffle(indexs)
    inputx = inputx[indexs]
    randnum = random.random()
    for index,val in zip(indexs,inputx):
        randnum -= val
        if randnum <= 0:
            return index

In [255]:
gameplay = GamePlay()

In [256]:
gameplay.make_move("77-47")
#gameplay.print_board()

gamearr = np.transpose(gameplay.get_board_arr(),(1,2,0)).reshape((1,10,9,14))
net_select = sess.run(net_softmax1,feed_dict={X1:gamearr,training:False})[0]
net_index = randargmax(net_select)
sel_x = net_index % 9
sel_y = net_index // 9

net_select = np.zeros(10 * 9)
net_select[net_index] = 1
net_select = net_select.reshape((10,9))

move_input = np.concatenate((gamearr,net_select.reshape(1,10,9,1)),axis=-1)
net_move = sess.run(net_softmax2,feed_dict={X2:move_input,training:False})[0]
net_index = randargmax(net_move)
mov_x = net_index % 9
mov_y = net_index // 9

gameplay.make_move("{}{}-{}{}".format(sel_x,sel_y,mov_x,mov_y))
gameplay.print_board()


0 砗─碼─象─士─将─士─象─┬─砗
  │  │  │  │＼│／│　│　│　│
1 ├─┼─┼─┼─※─┼─┼─┼─┤
  │　│　│　│／│＼│　│　│　│
2 ├─砲─┼─┼─┼─┼─碼─砲─┤
  │　│　│　│　│　│　│　│　│
3 卒─┼─卒─┼─卒─┼─卒─┼─卒
  │　│　│　│　│　│　│　│　│
4 ├─┴─┴─┴─┴─┴─┴─┴─┤
  │　                         　 │
5 ├─┬─┬─┬─┬─┬─┬─┬─┤
  │　│　│　│　│　│　│　│　│
6 兵─┼─兵─┼─兵─┼─兵─┼─兵
  │　│　│　│　│　│　│　│　│
7 ├─炮─┼─┼─炮─┼─┼─┼─┤
  │　│　│　│＼│／│　│　│　│
8 ├─┼─┼─┼─※─┼─┼─┼─┤
  │　│　│　│／│＼│　│　│　│
9 车─马─相─仕─帅─仕─相─马─车
  0   1   2   3   4   5   6   7   8



In [252]:
#gameplay.make_move("59-51")
#gameplay.print_board()
import time
begin = time.time()
gameplay = GamePlay()
for i in range(100):
    gamearr = np.transpose(gameplay.get_board_arr(),(1,2,0)).reshape((1,10,9,14))
    net_select = sess.run(net_softmax1,feed_dict={X1:gamearr,training:False})[0]
    net_index = randargmax(net_select)
    sel_x = net_index % 9
    sel_y = net_index // 9

    net_select = np.zeros(10 * 9)
    net_select[net_index] = 1
    net_select = net_select.reshape((10,9))

    move_input = np.concatenate((gamearr,net_select.reshape(1,10,9,1)),axis=-1)
    net_move = sess.run(net_softmax2,feed_dict={X2:move_input,training:False})[0]
    net_index = randargmax(net_move)
    mov_x = net_index % 9
    mov_y = net_index // 9
    side = gameplay.get_side()
    gameplay.make_move("{}{}-{}{}".format(sel_x,sel_y,mov_x,mov_y))
    if side == 'black':
        sel_y = 9 - sel_y
        mov_y = 9 - mov_y
    print("第{}手:{} {}{}-{}{} ".format(i + 1,side,sel_x,sel_y,mov_x,mov_y))
    gameplay.print_board()
end = time.time()

第1手:red 77-47 

0 砗─碼─象─士─将─士─象─碼─砗
  │  │  │  │＼│／│　│　│　│
1 ├─┼─┼─┼─※─┼─┼─┼─┤
  │　│　│　│／│＼│　│　│　│
2 ├─砲─┼─┼─┼─┼─┼─砲─┤
  │　│　│　│　│　│　│　│　│
3 卒─┼─卒─┼─卒─┼─卒─┼─卒
  │　│　│　│　│　│　│　│　│
4 ├─┴─┴─┴─┴─┴─┴─┴─┤
  │　                         　 │
5 ├─┬─┬─┬─┬─┬─┬─┬─┤
  │　│　│　│　│　│　│　│　│
6 兵─┼─兵─┼─兵─┼─兵─┼─兵
  │　│　│　│　│　│　│　│　│
7 ├─炮─┼─┼─炮─┼─┼─┼─┤
  │　│　│　│＼│／│　│　│　│
8 ├─┼─┼─┼─※─┼─┼─┼─┤
  │　│　│　│／│＼│　│　│　│
9 车─马─相─仕─帅─仕─相─马─车
  0   1   2   3   4   5   6   7   8

第2手:black 10-22 

0 砗─┬─象─士─将─士─象─碼─砗
  │  │  │  │＼│／│　│　│　│
1 ├─┼─┼─┼─※─┼─┼─┼─┤
  │　│　│　│／│＼│　│　│　│
2 ├─砲─碼─┼─┼─┼─┼─砲─┤
  │　│　│　│　│　│　│　│　│
3 卒─┼─卒─┼─卒─┼─卒─┼─卒
  │　│　│　│　│　│　│　│　│
4 ├─┴─┴─┴─┴─┴─┴─┴─┤
  │　                         　 │
5 ├─┬─┬─┬─┬─┬─┬─┬─┤
  │　│　│　│　│　│　│　│　│
6 兵─┼─兵─┼─兵─┼─兵─┼─兵
  │　│　│　│　│　│　│　│　│
7 ├─炮─┼─┼─炮─┼─┼─┼─┤
  │　│　│　│＼│／│　│　│　│
8 ├─┼─┼─┼─※─┼─┼─┼─┤
  │　│　│　│／│＼│　│　│　│
9 车─马─相─仕─帅─仕─相─马─车
  0   1   2   3   4   5   6   7   8

第3手:red 79-67 

0 砗─┬─象─士─将─士─象─碼─砗
  │  │  │  │＼│／│　│　│　│
1 ├─┼─┼─┼─※─┼─┼─┼─┤
  │　│　│　│／│＼│　│　│　│
2

第23手:red 79-71 

0 ┌─砗─┬─┬─将─士─象─┬─┐
  │  │  │  │＼│／│　│　│　│
1 ├─┼─炮─┼─士─┼─┼─车─┤
  │　│　│　│／│＼│　│　│　│
2 ├─砲─碼─┼─象─砲─┼─┼─碼
  │　│　│　│　│　│　│　│　│
3 卒─┼─┼─┼─卒─┼─卒─┼─卒
  │　│　│　│　│　│　│　│　│
4 ├─┴─卒─┴─┴─┴─┴─┴─┤
  │　                         　 │
5 ├─车─┬─┬─┬─┬─兵─┬─兵
  │　│　│　│　│　│　│　│　│
6 兵─┼─兵─┼─兵─┼─┼─┼─┤
  │　│　│　│　│　│　│　│　│
7 马─┼─┼─┼─炮─┼─马─┼─┤
  │　│　│　│＼│／│　│　│　│
8 ├─┼─┼─┼─仕─┼─┼─┼─┤
  │　│　│　│／│＼│　│　│　│
9 └─┴─相─仕─帅─┴─相─┴─┘
  0   1   2   3   4   5   6   7   8

第24手:black 10-11 

0 ┌─┬─┬─┬─将─士─象─┬─┐
  │  │  │  │＼│／│　│　│　│
1 ├─砗─炮─┼─士─┼─┼─车─┤
  │　│　│　│／│＼│　│　│　│
2 ├─砲─碼─┼─象─砲─┼─┼─碼
  │　│　│　│　│　│　│　│　│
3 卒─┼─┼─┼─卒─┼─卒─┼─卒
  │　│　│　│　│　│　│　│　│
4 ├─┴─卒─┴─┴─┴─┴─┴─┤
  │　                         　 │
5 ├─车─┬─┬─┬─┬─兵─┬─兵
  │　│　│　│　│　│　│　│　│
6 兵─┼─兵─┼─兵─┼─┼─┼─┤
  │　│　│　│　│　│　│　│　│
7 马─┼─┼─┼─炮─┼─马─┼─┤
  │　│　│　│＼│／│　│　│　│
8 ├─┼─┼─┼─仕─┼─┼─┼─┤
  │　│　│　│／│＼│　│　│　│
9 └─┴─相─仕─帅─┴─相─┴─┘
  0   1   2   3   4   5   6   7   8

第25手:red 47-37 

0 ┌─┬─┬─┬─将─士─象─┬─┐
  │  │  │  │＼│／│　│　│　│
1 ├─砗─炮─┼─士─┼─┼─车─┤
  │　│　│　│／│＼│　│　│　

第45手:red 72-52 

0 ┌─┬─┬─┬─将─士─┬─┬─┐
  │  │  │  │＼│／│　│　│　│
1 ├─┼─┼─┼─士─┼─┼─┼─┤
  │　│　│　│／│＼│　│　│　│
2 ├─┼─碼─┼─象─车─┼─┼─象
  │　│　│　│　│　│　│　│　│
3 卒─┼─┼─┼─卒─┼─┼─┼─卒
  │　│　│　│　│　│　│　│　│
4 ├─┴─┴─砗─┴─┴─┴─┴─┤
  │　                         　 │
5 ├─┬─┬─┬─车─┬─兵─┬─兵
  │　│　│　│　│　│　│　│　│
6 兵─┼─兵─┼─┼─┼─砲─┼─┤
  │　│　│　│　│　│　│　│　│
7 ├─┼─┼─┼─炮─┼─┼─┼─┤
  │　│　│　│＼│／│　│　│　│
8 ├─┼─马─┼─仕─┼─┼─┼─┤
  │　│　│　│／│＼│　│　│　│
9 └─┴─相─仕───帅─相─┴─┘
  0   1   2   3   4   5   6   7   8

第46手:black 34-39 

0 ┌─┬─┬─┬─将─士─┬─┬─┐
  │  │  │  │＼│／│　│　│　│
1 ├─┼─┼─┼─士─┼─┼─┼─┤
  │　│　│　│／│＼│　│　│　│
2 ├─┼─碼─┼─象─车─┼─┼─象
  │　│　│　│　│　│　│　│　│
3 卒─┼─┼─┼─卒─┼─┼─┼─卒
  │　│　│　│　│　│　│　│　│
4 ├─┴─┴─┴─┴─┴─┴─┴─┤
  │　                         　 │
5 ├─┬─┬─┬─车─┬─兵─┬─兵
  │　│　│　│　│　│　│　│　│
6 兵─┼─兵─┼─┼─┼─砲─┼─┤
  │　│　│　│　│　│　│　│　│
7 ├─┼─┼─┼─炮─┼─┼─┼─┤
  │　│　│　│＼│／│　│　│　│
8 ├─┼─马─┼─仕─┼─┼─┼─┤
  │　│　│　│／│＼│　│　│　│
9 └─┴─相─砗───帅─相─┴─┘
  0   1   2   3   4   5   6   7   8

第47手:red 48-39 

0 ┌─┬─┬─┬─将─士─┬─┬─┐
  │  │  │  │＼│／│　│　│　│
1 ├─┼─┼─┼─士─┼─┼─┼─┤
  │　│　│　│／│＼│　│　│　

第69手:red 25-24 

0 ┌─┬─┬─┬─将─┬─┬─┬─┐
  │  │  │  │＼│／│　│　│　│
1 ├─┼─┼─┼─士─┼─┼─砲─┤
  │　│　│　│／│＼│　│　│　│
2 ├─┼─┼─碼─象─士─┼─┼─┤
  │　│　│　│　│　│　│　│　│
3 ├─┼─┼─┼─┼─┼─┼─┼─卒
  │　│　│　│　│　│　│　│　│
4 卒─┴─兵─炮─┴─┴─象─┴─┤
  │　                         　 │
5 ├─┬─┬─车─┬─┬─兵─┬─兵
  │　│　│　│　│　│　│　│　│
6 兵─┼─┼─┼─┼─┼─┼─┼─┤
  │　│　│　│　│　│　│　│　│
7 ├─┼─┼─┼─┼─┼─┼─┼─┤
  │　│　│　│＼│／│　│　│　│
8 ├─┼─┼─┼─※─帅─┼─┼─┤
  │　│　│　│／│＼│　│　│　│
9 └─┴─相─仕───┴─相─┴─┘
  0   1   2   3   4   5   6   7   8

第70手:black 32-24 

0 ┌─┬─┬─┬─将─┬─┬─┬─┐
  │  │  │  │＼│／│　│　│　│
1 ├─┼─┼─┼─士─┼─┼─砲─┤
  │　│　│　│／│＼│　│　│　│
2 ├─┼─┼─┼─象─士─┼─┼─┤
  │　│　│　│　│　│　│　│　│
3 ├─┼─┼─┼─┼─┼─┼─┼─卒
  │　│　│　│　│　│　│　│　│
4 卒─┴─碼─炮─┴─┴─象─┴─┤
  │　                         　 │
5 ├─┬─┬─车─┬─┬─兵─┬─兵
  │　│　│　│　│　│　│　│　│
6 兵─┼─┼─┼─┼─┼─┼─┼─┤
  │　│　│　│　│　│　│　│　│
7 ├─┼─┼─┼─┼─┼─┼─┼─┤
  │　│　│　│＼│／│　│　│　│
8 ├─┼─┼─┼─※─帅─┼─┼─┤
  │　│　│　│／│＼│　│　│　│
9 └─┴─相─仕───┴─相─┴─┘
  0   1   2   3   4   5   6   7   8

第71手:red 34-44 

0 ┌─┬─┬─┬─将─┬─┬─┬─┐
  │  │  │  │＼│／│　│　│　│
1 ├─┼─┼─┼─士─┼─┼─砲─┤
  │　│　│　│／│＼│　│　│　

第92手:black 51-41 

0 ┌─┬─┬─将───士─┬─┬─┐
  │  │  │  │＼│／│　│　│　│
1 ├─┼─┼─┼─砲─┼─┼─┼─┤
  │　│　│　│／│＼│　│　│　│
2 ├─┼─┼─┼─┼─士─┼─┼─┤
  │　│　│　│　│　│　│　│　│
3 ├─┼─┼─┼─┼─┼─┼─┼─卒
  │　│　│　│　│　│　│　│　│
4 ├─┴─碼─┴─┴─┴─象─┴─┤
  │　                         　 │
5 ├─卒─┬─┬─┬─┬─┬─┬─兵
  │　│　│　│　│　│　│　│　│
6 兵─┼─┼─┼─┼─┼─┼─车─┤
  │　│　│　│　│　│　│　│　│
7 ├─┼─┼─┼─相─┼─┼─┼─┤
  │　│　│　│＼│／│　│　│　│
8 ├─┼─┼─┼─※─┼─┼─┼─┤
  │　│　│　│／│＼│　│　│　│
9 └─┴─相─仕─帅─┴─┴─┴─炮
  0   1   2   3   4   5   6   7   8

第93手:red 49-59 

0 ┌─┬─┬─将───士─┬─┬─┐
  │  │  │  │＼│／│　│　│　│
1 ├─┼─┼─┼─砲─┼─┼─┼─┤
  │　│　│　│／│＼│　│　│　│
2 ├─┼─┼─┼─┼─士─┼─┼─┤
  │　│　│　│　│　│　│　│　│
3 ├─┼─┼─┼─┼─┼─┼─┼─卒
  │　│　│　│　│　│　│　│　│
4 ├─┴─碼─┴─┴─┴─象─┴─┤
  │　                         　 │
5 ├─卒─┬─┬─┬─┬─┬─┬─兵
  │　│　│　│　│　│　│　│　│
6 兵─┼─┼─┼─┼─┼─┼─车─┤
  │　│　│　│　│　│　│　│　│
7 ├─┼─┼─┼─相─┼─┼─┼─┤
  │　│　│　│＼│／│　│　│　│
8 ├─┼─┼─┼─※─┼─┼─┼─┤
  │　│　│　│／│＼│　│　│　│
9 └─┴─相─仕───帅─┴─┴─炮
  0   1   2   3   4   5   6   7   8

第94手:black 24-45 

0 ┌─┬─┬─将───士─┬─┬─┐
  │  │  │  │＼│／│　│　│　│
1 ├─┼─┼─┼─砲─┼─┼─┼─┤
  │　│　│　│／│＼│　│　

In [253]:
end - begin

1.8445706367492676

In [233]:
net_select = sess.run(net_softmax1,feed_dict={X1:gamearr,training:False})[0]